# Latent Space Clustering and Generating

In [12]:
import numpy as np
import utils
from data import loader
import math
from model.rvae import RVAE
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
import matplotlib.pyplot as plt

%matplotlib widget

In [13]:
config = utils.read_cfg('../config/template.yaml')
train, test_scenarios = loader.load(config)
test = test_scenarios['ctu-1-2-9']

train_norm, train_bot = utils.split_mal_norm(train)
count_x = math.floor(train_norm.shape[0] / 5)
train_norm = utils.remove_ylabel(train_norm)
train_norm = utils.reshape_for_rnn(train_norm.values)

train_bot = utils.remove_ylabel(train_bot)
train_bot = utils.reshape_for_rnn(train_bot.values)

test_norm, test_bot = utils.split_mal_norm(test)
test_norm = utils.remove_ylabel(test_norm)
test_norm = utils.reshape_for_rnn(test_norm.values)
test_bot = utils.remove_ylabel(test_bot)
test_bot = utils.reshape_for_rnn(test_bot.values)

feature_dim = train_norm.shape

print('jo')

jo


In [14]:
encoder_inputs = keras.Input(shape=(feature_dim[1], feature_dim[2]))
encoder_layers = layers.GRU(units=10)(encoder_inputs)
encoder_layers = layers.LeakyReLU(alpha=0.01)(encoder_layers)

# latent dim 3 for visualisation
latent_inputs = keras.Input(shape=(5))
decoder_layers = layers.Dense(feature_dim[2] * feature_dim[1])(latent_inputs)
decoder_layers = layers.Reshape((feature_dim[1], feature_dim[2]))(decoder_layers)
decoder_layers = layers.GRU(units=10, activation='relu', return_sequences=True)(decoder_layers)
decoder_layers = layers.Dense(5)(decoder_layers)

vae = RVAE(encoder_inputs, encoder_layers, latent_inputs, decoder_layers)
vae.compile(optimizer=keras.optimizers.Adam())

vae.fit(train_norm, epochs=500, batch_size=128)

Epoch 1/500
14/14 [==============================] - 4s 24ms/step - loss: 14.5728 - reconstruction_loss: 13.7990 - kl_loss: 0.0997 - z_mean: 0.0773 - z_log_var: -0.0853
Epoch 2/500
14/14 [==============================] - 0s 22ms/step - loss: 11.6289 - reconstruction_loss: 11.2268 - kl_loss: 0.0852 - z_mean: 0.0744 - z_log_var: -0.0844
Epoch 3/500
14/14 [==============================] - 0s 22ms/step - loss: 9.8940 - reconstruction_loss: 9.4892 - kl_loss: 0.0767 - z_mean: 0.0702 - z_log_var: -0.0845
Epoch 4/500
14/14 [==============================] - 0s 22ms/step - loss: 8.8520 - reconstruction_loss: 8.5606 - kl_loss: 0.0838 - z_mean: 0.0647 - z_log_var: -0.0865
Epoch 5/500
14/14 [==============================] - 0s 26ms/step - loss: 8.3030 - reconstruction_loss: 8.1402 - kl_loss: 0.0876 - z_mean: 0.0599 - z_log_var: -0.0897
Epoch 6/500
14/14 [==============================] - 0s 24ms/step - loss: 7.9039 - reconstruction_loss: 7.7430 - kl_loss: 0.0865 - z_mean: 0.0561 - z_log_var: -0

In [15]:
mu, log_sig, z = vae.encoder.predict(train_norm)
nu_mal, log_sig_mal, z_mal = vae.encoder.predict(train_bot)

In [16]:
# fig = plt.figure(figsize=(20, 10))
# ax = fig.add_subplot(projection='3d')
# plt.scatter(z[:, 0], z[:, 1], color='green')
# plt.scatter(z_mal[:, 0], z_mal[:, 1], color='red')
# plt.show()

In [17]:
# plt.figure(figsize=(20, 10))
model = DBSCAN(eps=0.5, min_samples=2)
# model = OPTICS(eps=0.1, min_samples=5)
# fit model and predict clusters
yhat = model.fit_predict(z)
# retrieve unique clusters
clusters = np.unique(yhat)
# create scatter plot for samples from each cluster
augmentations = []
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(yhat == cluster)
    cluster_data = z[row_ix]
    # if cluster_data.shape[0]:
    mean = np.mean(cluster_data, axis=(0))
    std = np.std(cluster_data, axis=(0))
    sample = np.random.normal(mean, std, size=(5, mean.shape[0]))
    # dim_1 = np.random.normal(mean[0], std[0], size=(5))
    # dim_2 = np.random.normal(mean[1], std[1], size=(5))
    # _, _, zz = vae.encoder.predict(vae.decoder.predict(np.array([dim_1, dim_2]).T))
    augmentations.append(sample)
    # plt.scatter(sample[:, 0], sample[:, 1], marker='x', s=50)
    # plt.scatter(zz[:, 0], zz[:, 1], marker='d', s=50)
    # plt.scatter(z[row_ix, 0], z[row_ix, 1])
    # else:
    # plt.scatter(z[row_ix, 0], z[row_ix, 1], alpha=0.3)

# create scatter of these samples

# plt.scatter(z_mal[:,0], z_mal[:,1], color='red')
# show the plot
augmentations = np.concatenate(augmentations)
# plt.show()

In [18]:
generated_data = vae.decoder.predict(augmentations)
_, _, zz = vae.encoder.predict(generated_data)
# generated_data = np.maximum(generated_data, np.zeros(generated_data.shape))
augmented_data = np.concatenate([train_norm, generated_data])

In [19]:
vae2 = RVAE(encoder_inputs, encoder_layers, latent_inputs, decoder_layers)
vae2.compile(optimizer=keras.optimizers.Adam())

vae2.fit(augmented_data, epochs=500, batch_size=128)

Epoch 1/500
21/21 [==============================] - 3s 16ms/step - loss: -0.7722 - reconstruction_loss: -0.5954 - kl_loss: 1.1846e-04 - z_mean: 7.8741e-04 - z_log_var: -0.0024
Epoch 2/500
21/21 [==============================] - 0s 16ms/step - loss: -0.5957 - reconstruction_loss: -0.6277 - kl_loss: 3.9982e-04 - z_mean: -2.8930e-04 - z_log_var: -0.0083
Epoch 3/500
21/21 [==============================] - 0s 16ms/step - loss: -0.6499 - reconstruction_loss: -0.6602 - kl_loss: 8.5749e-04 - z_mean: -0.0015 - z_log_var: -0.0126
Epoch 4/500
21/21 [==============================] - 0s 17ms/step - loss: -0.5888 - reconstruction_loss: -0.5826 - kl_loss: 0.0011 - z_mean: -0.0026 - z_log_var: -0.0155
Epoch 5/500
21/21 [==============================] - 0s 17ms/step - loss: -0.6074 - reconstruction_loss: -0.6157 - kl_loss: 0.0011 - z_mean: -0.0032 - z_log_var: -0.0175
Epoch 6/500
21/21 [==============================] - 0s 17ms/step - loss: -0.7440 - reconstruction_loss: -0.6157 - kl_loss: 0.0013 

In [20]:
mu2, log_sig2, z2 = vae2.encoder.predict(train_norm)
nu_mal2, log_sig_mal2, z_mal2 = vae2.encoder.predict(train_bot)

In [21]:
# fig = plt.figure(figsize=(20, 10))
# ax = fig.add_subplot(projection='3d')
# plt.scatter(z2[:, 0], z2[:, 1], color='green')
# plt.scatter(z_mal2[:, 0], z_mal2[:, 1], color='red')
# plt.show()

In [22]:
from downstream.kernel_density import KDEDownstream

x = np.concatenate([train_norm, train_bot])

err_1,_,_ = utils.pred(vae, x, axis=(1, 2))
err_2,_,_ = utils.pred(vae2, x, axis=(1, 2))
y = np.concatenate((np.zeros(train_norm.shape[0]), np.ones(train_bot.shape[0])), axis=None)

m1 = KDEDownstream()
m1.fit(err_1, y)

m2 = KDEDownstream()
m2.fit(err_2, y)

utils.simple_eval(test, vae, m1)
utils.simple_eval(test, vae2, m2)

#normal: 422 / #bot: 378
prec:0.883177570093458
rec:1.0
f1: 0.9379652605459057
[[372  50]
 [  0 378]]
#normal: 422 / #bot: 378
prec:0.8894117647058823
rec:1.0
f1: 0.9414694894146949
[[375  47]
 [  0 378]]
